In [1]:
import json
import base64
import qrcode
import requests
import datetime
import uuid
import matplotlib.pyplot as plt
from pymongo import MongoClient
from typing import Optional, List
from didcomm.common.types import DID, VerificationMethodType, VerificationMaterial, VerificationMaterialFormat
from didcomm.did_doc.did_doc import DIDDoc, VerificationMethod, DIDCommService
from didcomm.did_doc.did_resolver import DIDResolver
from didcomm.message import Message, FromPrior
from didcomm.secrets.secrets_resolver_demo import SecretsResolverDemo
from didcomm.unpack import unpack, UnpackResult
from didcomm.common.resolvers import ResolversConfig
from didcomm.pack_encrypted import pack_encrypted, PackEncryptedConfig, PackEncryptedResult
from peerdid.core.did_doc_types import DIDCommServicePeerDID
from didcomm.secrets.secrets_util import generate_x25519_keys_as_jwk_dict, generate_ed25519_keys_as_jwk_dict, jwk_to_secret
from peerdid import peer_did
from peerdid.did_doc import DIDDocPeerDID
from peerdid.types import VerificationMaterialAuthentication, VerificationMethodTypeAuthentication, VerificationMaterialAgreement, VerificationMethodTypeAgreement, VerificationMaterialFormatPeerDID

In [2]:
secrets_resolver = SecretsResolverDemo()

In [3]:
class DIDResolverPeerDID(DIDResolver):
    async def resolve(self, did: DID) -> DIDDoc:
        did_doc_json = peer_did.resolve_peer_did(did, format = VerificationMaterialFormatPeerDID.JWK)
        did_doc = DIDDocPeerDID.from_json(did_doc_json)

        return DIDDoc(
            did=did_doc.did,
            key_agreement_kids = did_doc.agreement_kids,
            authentication_kids = did_doc.auth_kids,
            verification_methods = [
                VerificationMethod(
                    id = m.id,
                    type = VerificationMethodType.JSON_WEB_KEY_2020,
                    controller = m.controller,
                    verification_material = VerificationMaterial(
                        format = VerificationMaterialFormat.JWK,
                        value = json.dumps(m.ver_material.value)
                    )
                )
                for m in did_doc.authentication + did_doc.key_agreement
            ],
            didcomm_services = [
                DIDCommService(
                    id = s.id,
                    service_endpoint = s.service_endpoint,
                    routing_keys = s.routing_keys,
                    accept = s.accept
                )
                for s in did_doc.service
                if isinstance(s, DIDCommServicePeerDID)
            ] if did_doc.service else []
        )

In [4]:
async def create_peer_did(self,
                        auth_keys_count: int = 1,
                        agreement_keys_count: int = 1,
                        service_endpoint: Optional[str] = None,
                        service_routing_keys: Optional[List[str]] = None
                        ) -> str:
        # 1. generate keys in JWK format
        agreem_keys = [generate_x25519_keys_as_jwk_dict() for _ in range(agreement_keys_count)]
        auth_keys = [generate_ed25519_keys_as_jwk_dict() for _ in range(auth_keys_count)]

        # 2. prepare the keys for peer DID lib
        agreem_keys_peer_did = [
            VerificationMaterialAgreement(
                type=VerificationMethodTypeAgreement.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in agreem_keys
        ]
        auth_keys_peer_did = [
            VerificationMaterialAuthentication(
                type=VerificationMethodTypeAuthentication.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in auth_keys
        ]

        # 3. generate service
        service = None
        if service_endpoint:
            service = json.dumps(
                DIDCommServicePeerDID(
                    id="new-id",
                    service_endpoint=service_endpoint, routing_keys=service_routing_keys,
                    accept=["didcomm/v2"]
                ).to_dict()
            )

        # 4. call peer DID lib
        # if we have just one key (auth), then use numalg0 algorithm
        # otherwise use numalg2 algorithm
        if len(auth_keys_peer_did) == 1 and not agreem_keys_peer_did and not service:
            did = peer_did.create_peer_did_numalgo_0(auth_keys_peer_did[0])
        else:
            did = peer_did.create_peer_did_numalgo_2(
                encryption_keys=agreem_keys_peer_did,
                signing_keys=auth_keys_peer_did,
                service=service,
            )

        # 5. set KIDs as in DID DOC for secrets and store the secret in the secrets resolver
        did_doc = DIDDocPeerDID.from_json(peer_did.resolve_peer_did(did))
        for auth_key, kid in zip(auth_keys, did_doc.auth_kids):
            private_key = auth_key[0]
            private_key["kid"] = kid
            print(private_key)
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        for agreem_key, kid in zip(agreem_keys, did_doc.agreement_kids):
            private_key = agreem_key[0]
            private_key["kid"] = kid
            print(private_key)
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        return did


### Alice get OOB from Mediator

In [5]:
oob_url = requests.get("http://rootsid-mediator:8000/oob_url").text
#oob_url = requests.get("http://127.0.0.1:8000/oob_url").text
#oob_url = requests.get("https://mediator.rootsid.cloud/oob_url").text

In [6]:
print(oob_url)

http://rootsid-mediator:8000?_oob=eyJ0eXBlIjoiaHR0cHM6Ly9kaWRjb21tLm9yZy9vdXQtb2YtYmFuZC8yLjAvaW52aXRhdGlvbiIsImlkIjoiYjFkN2Y0OWYtYzUyMS00NDk0LTg3MzQtYzAwZDhmY2M2NGFlIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNmdVlvZUU4QXJvclJuVkVIRzZnMVRVYjlZNjZLZkczNVdYcmdEbnJmWGtkVi5WejZNa3ZaVlp3SzlSVGtuTThEN0JKM0Q3aVphdTNBSFlKV0M2a1VLcWtvZEtobVdYLlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd09pOHZjbTl2ZEhOcFpDMXRaV1JwWVhSdmNqbzRNREF3SWl3aVlTSTZXeUprYVdSamIyMXRMM1l5SWwxOSIsImJvZHkiOnsiZ29hbF9jb2RlIjoicmVxdWVzdC1tZWRpYXRlIiwiZ29hbCI6IlJlcXVlc3RNZWRpYXRlIiwiYWNjZXB0IjpbImRpZGNvbW0vdjIiLCJkaWRjb21tL2FpcDI7ZW52PXJmYzU4NyJdfX0


In [7]:
received_msg_encoded = oob_url.split("=")[1]
received_msg_decoded = json.loads(str(base64.urlsafe_b64decode(received_msg_encoded + "=="), "utf-8"))
alice_did_for_mediator = await create_peer_did(1,1)
print("Alice's DID:", alice_did_for_mediator)

{'crv': 'Ed25519', 'x': 'ahRGFauYqLNRt713YmCqkTuwIj6o15WJfmwC6wXmGZQ', 'd': 'bQfbuHb9By5CUSToXBh_2n2-4-8v9nVFtWixtib_jaE', 'kty': 'OKP', 'kid': 'did:peer:2.Ez6LSi6SDgieS8H7FC3UREj2H1iYVCL7S9M8iTCPEmoGtiutJ.Vz6MkmbMC51u4xFNabEXoB32LP9aqJt6f39A3mmmqwuQYMhVm#6MkmbMC51u4xFNabEXoB32LP9aqJt6f39A3mmmqwuQYMhVm'}
{'crv': 'X25519', 'x': 'X10VSyUGc2y_uGQU1PZPglsQZD4l73-TBCG11eSxSGc', 'd': 'sP7MbcTnExW9NW9ucc_A7GJ41-shNo2zzAW9PTFonlk', 'kty': 'OKP', 'kid': 'did:peer:2.Ez6LSi6SDgieS8H7FC3UREj2H1iYVCL7S9M8iTCPEmoGtiutJ.Vz6MkmbMC51u4xFNabEXoB32LP9aqJt6f39A3mmmqwuQYMhVm#6LSi6SDgieS8H7FC3UREj2H1iYVCL7S9M8iTCPEmoGtiutJ'}
Alice's DID: did:peer:2.Ez6LSi6SDgieS8H7FC3UREj2H1iYVCL7S9M8iTCPEmoGtiutJ.Vz6MkmbMC51u4xFNabEXoB32LP9aqJt6f39A3mmmqwuQYMhVm


### Alice request mediate

In [8]:
alice_mediate_grant = Message(
    custom_headers = [{"return_route": "all"}],
    id = str(uuid.uuid4()),
    type = "https://didcomm.org/coordinate-mediation/2.0/mediate-request",
    body = {}
)

In [9]:
alice_mediate_grant_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_mediate_grant,
    frm = alice_did_for_mediator,
    to = received_msg_decoded["from"],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
print(alice_mediate_grant_packed)

PackEncryptedResult(packed_msg='{"protected":"eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC0xUFUrQTI1NktXIiwiZW5jIjoiQTI1NkNCQy1IUzUxMiIsImFwdSI6IlpHbGtPbkJsWlhJNk1pNUZlalpNVTJrMlUwUm5hV1ZUT0VnM1JrTXpWVkpGYWpKSU1XbFpWa05NTjFNNVRUaHBWRU5RUlcxdlIzUnBkWFJLTGxaNk5rMXJiV0pOUXpVeGRUUjRSazVoWWtWWWIwSXpNa3hRT1dGeFNuUTJaak01UVROdGJXMXhkM1ZSV1Uxb1ZtMGpOa3hUYVRaVFJHZHBaVk00U0RkR1F6TlZVa1ZxTWtneGFWbFdRMHczVXpsTk9HbFVRMUJGYlc5SGRHbDFkRW8iLCJhcHYiOiJUVTRLbXh3eDB3RWxBb0pac21FMG1HZ05GWkI5LVFfWDJCdG1RQzdtRllRIiwic2tpZCI6ImRpZDpwZWVyOjIuRXo2TFNpNlNEZ2llUzhIN0ZDM1VSRWoySDFpWVZDTDdTOU04aVRDUEVtb0d0aXV0Si5WejZNa21iTUM1MXU0eEZOYWJFWG9CMzJMUDlhcUp0NmYzOUEzbW1tcXd1UVlNaFZtIzZMU2k2U0RnaWVTOEg3RkMzVVJFajJIMWlZVkNMN1M5TThpVENQRW1vR3RpdXRKIiwiZXBrIjp7ImNydiI6IlgyNTUxOSIsIngiOiJ1QjJfQWRCTWZSSjEzR2R5Z0lFUmFUYVU2SGh5SFBFRF94ZDlEWnlsYkI0Iiwia3R5IjoiT0tQIn19","recipients":[{"header":{"kid":"did:peer:2.Ez6LSfuYoeE8ArorRnVEHG6g1TUb9Y66KfG35WXrgDnrfXkdV.Vz6MkvZVZwK9RTknM8D7BJ3D7iZau3AHYJWC6kU

### Sending to Mediator

In [10]:
mediator_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))
mediator_endpoint = mediator_did_doc["service"][0]["serviceEndpoint"]
print(mediator_did_doc)
print(mediator_endpoint)

{'id': 'did:peer:2.Ez6LSfuYoeE8ArorRnVEHG6g1TUb9Y66KfG35WXrgDnrfXkdV.Vz6MkvZVZwK9RTknM8D7BJ3D7iZau3AHYJWC6kUKqkodKhmWX.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vcm9vdHNpZC1tZWRpYXRvcjo4MDAwIiwiYSI6WyJkaWRjb21tL3YyIl19', 'authentication': [{'id': 'did:peer:2.Ez6LSfuYoeE8ArorRnVEHG6g1TUb9Y66KfG35WXrgDnrfXkdV.Vz6MkvZVZwK9RTknM8D7BJ3D7iZau3AHYJWC6kUKqkodKhmWX.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vcm9vdHNpZC1tZWRpYXRvcjo4MDAwIiwiYSI6WyJkaWRjb21tL3YyIl19#6MkvZVZwK9RTknM8D7BJ3D7iZau3AHYJWC6kUKqkodKhmWX', 'type': 'Ed25519VerificationKey2020', 'controller': 'did:peer:2.Ez6LSfuYoeE8ArorRnVEHG6g1TUb9Y66KfG35WXrgDnrfXkdV.Vz6MkvZVZwK9RTknM8D7BJ3D7iZau3AHYJWC6kUKqkodKhmWX.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vcm9vdHNpZC1tZWRpYXRvcjo4MDAwIiwiYSI6WyJkaWRjb21tL3YyIl19', 'publicKeyMultibase': 'z6MkvZVZwK9RTknM8D7BJ3D7iZau3AHYJWC6kUKqkodKhmWX'}], 'keyAgreement': [{'id': 'did:peer:2.Ez6LSfuYoeE8ArorRnVEHG6g1TUb9Y66KfG35WXrgDnrfXkdV.Vz6MkvZVZwK9RTknM8D7BJ3D7iZau3AHYJWC6kUKqkodK

In [11]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(mediator_endpoint, headers=headers, data = alice_mediate_grant_packed.packed_msg)


In [12]:
print(resp.json())

{"protected":"eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC0xUFUrQTI1NktXIiwiZW5jIjoiQTI1NkNCQy1IUzUxMiIsImFwdSI6IlpHbGtPbkJsWlhJNk1pNUZlalpNVTI4M2JubG1kMWt6YlRkM1UxRjJRekZOTTFVMU5sUm9kbXBaYURVMFlrdEtUVU5ZYVhONlRuZEhNMlp0TGxaNk5rMXJiMDVaYWpGMGQyTlZaSFJJWWs1RFV6WlpjMUprZEZGdVZYWnBXbFkwVVVoRFFXNXJla3BLTW1seFZVTXVVMlY1U25CYVEwazJTVzAxYkdSNU1YQmFRMGx6U1c1UmFVOXBTbXRpVTBselNXNU5hVTlwU205a1NGSjNUMms0ZG1OdE9YWmtTRTV3V2tNeGRGcFhVbkJaV0ZKMlkycHZORTFFUVhkSmFYZHBXVk5KTmxkNVNtdGhWMUpxWWpJeGRFd3pXWGxKYkRFNUl6Wk1VMjgzYm5sbWQxa3piVGQzVTFGMlF6Rk5NMVUxTmxSb2RtcFphRFUwWWt0S1RVTllhWE42VG5kSE0yWnQiLCJhcHYiOiJpWlpVYklVZ25fWVlCeFlxNHdxaXR5N0s0aVZWVFJwUjZvaC1BSEEteFVNIiwic2tpZCI6ImRpZDpwZWVyOjIuRXo2TFNvN255ZndZM203d1NRdkMxTTNVNTZUaHZqWWg1NGJLSk1DWGlzek53RzNmbS5WejZNa29OWWoxdHdjVWR0SGJOQ1M2WXNSZHRRblV2aVpWNFFIQ0Fua3pKSjJpcVVDLlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd09pOHZjbTl2ZEhOcFpDMXRaV1JwWVhSdmNqbzRNREF3SWl3aVlTSTZXeUprYVdSamIyMXRMM1l5SWwxOSM2TFNvN255ZndZM2

### Get mediator grant response

In [13]:
mediator_unpack_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)
print(mediator_unpack_msg.message.type)
print("Routing Key: " + mediator_unpack_msg.message.body["routing_did"])
print("Mediator new DID: " + mediator_unpack_msg.message.from_prior.sub)
# mediator rotated did
mediator_routing_key = mediator_unpack_msg.message.body["routing_did"]
mediator_did = mediator_unpack_msg.message.from_prior.sub

https://didcomm.org/coordinate-mediation/2.0/mediate-grant
Routing Key: did:peer:2.Ez6LSh86cZe85p4LLsNc28nAp9rtLvg2tdS7KA9cYcB9DpnNY.Vz6Mknau6GqSR279G1akj6imwmXGqxLk5sskPZcnnT83ym1Wa.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vcm9vdHNpZC1tZWRpYXRvcjo4MDAwIiwiYSI6WyJkaWRjb21tL3YyIl19
Mediator new DID: did:peer:2.Ez6LSo7nyfwY3m7wSQvC1M3U56ThvjYh54bKJMCXiszNwG3fm.Vz6MkoNYj1twcUdtHbNCS6YsRdtQnUviZV4QHCAnkzJJ2iqUC.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vcm9vdHNpZC1tZWRpYXRvcjo4MDAwIiwiYSI6WyJkaWRjb21tL3YyIl19


### Keylist update

Alice create a new DID to share with  other peers using mediator routing keys 

In [14]:
#alice_did_new = await create_peer_did(1, 1, service_endpoint=mediator_endpoint, service_routing_keys=[mediator_routing_key])
alice_did_for_bob = await create_peer_did(1, 1, service_endpoint=[{"uri": mediator_routing_key}])

print("Alice's DID for Bob:", alice_did_for_bob)

{'crv': 'Ed25519', 'x': 'VbWFIhfT1aUvzxVcZJlGm40KZ7iw3swu2BfKUiS32Q0', 'd': 'fjpPxHmNxBCYOj2Mbi8Zdqw4HKCxaNBXcINIfQNrlhs', 'kty': 'OKP', 'kid': 'did:peer:2.Ez6LSqrUDvHgg8WbUH2gRfgGdNNhABreeuCYmBRTjMDe1AmVL.Vz6MkkDqEhfa7BUpaKsHM4Cph7YnjndRZfLJdqMtixA4jUV3r.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOlt7InVyaSI6ImRpZDpwZWVyOjIuRXo2TFNoODZjWmU4NXA0TExzTmMyOG5BcDlydEx2ZzJ0ZFM3S0E5Y1ljQjlEcG5OWS5WejZNa25hdTZHcVNSMjc5RzFha2o2aW13bVhHcXhMazVzc2tQWmNublQ4M3ltMVdhLlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd09pOHZjbTl2ZEhOcFpDMXRaV1JwWVhSdmNqbzRNREF3SWl3aVlTSTZXeUprYVdSamIyMXRMM1l5SWwxOSJ9XSwiYSI6WyJkaWRjb21tL3YyIl19#6MkkDqEhfa7BUpaKsHM4Cph7YnjndRZfLJdqMtixA4jUV3r'}
{'crv': 'X25519', 'x': '0qa2ljtJpd5hOoV-1TP9NUrQJdeT4HyzQq2onFVbjyM', 'd': 'aEoknbH22Fnm9cCdVJRbNpryELRBSX8nGGerzVcYh0o', 'kty': 'OKP', 'kid': 'did:peer:2.Ez6LSqrUDvHgg8WbUH2gRfgGdNNhABreeuCYmBRTjMDe1AmVL.Vz6MkkDqEhfa7BUpaKsHM4Cph7YnjndRZfLJdqMtixA4jUV3r.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOlt7InVyaSI6ImRpZDpwZWVyOjIuRXo2

Alice create and send the keylist update message

In [15]:
alice_keylist_update = Message(
    id = "unique-id-293e9a922efff",
    type = "https://didcomm.org/coordinate-mediation/2.0/keylist-update",
    body = {
        "updates":[
        {
            "recipient_did": alice_did_for_bob,
            "action": "add"
        }
    ]
    }
)

In [16]:
alice_keylist_update_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_keylist_update,
    frm = alice_did_for_mediator,
    to = mediator_did,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

In [17]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(mediator_endpoint, headers=headers, data = alice_keylist_update_packed_msg.packed_msg)

In [18]:
mediator_unpack_msg2 = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)
print(mediator_unpack_msg2.message.type)
print(mediator_unpack_msg2.message.body)


https://didcomm.org/coordinate-mediation/2.0/keylist-update-response
{'updated': [{'recipient_did': 'did:peer:2.Ez6LSqrUDvHgg8WbUH2gRfgGdNNhABreeuCYmBRTjMDe1AmVL.Vz6MkkDqEhfa7BUpaKsHM4Cph7YnjndRZfLJdqMtixA4jUV3r.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOlt7InVyaSI6ImRpZDpwZWVyOjIuRXo2TFNoODZjWmU4NXA0TExzTmMyOG5BcDlydEx2ZzJ0ZFM3S0E5Y1ljQjlEcG5OWS5WejZNa25hdTZHcVNSMjc5RzFha2o2aW13bVhHcXhMazVzc2tQWmNublQ4M3ltMVdhLlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd09pOHZjbTl2ZEhOcFpDMXRaV1JwWVhSdmNqbzRNREF3SWl3aVlTSTZXeUprYVdSamIyMXRMM1l5SWwxOSJ9XSwiYSI6WyJkaWRjb21tL3YyIl19', 'action': 'add', 'result': 'success'}]}


### Alice can now send her DID to Bob.

### Alice check status of queued messages from the Mediator

In [19]:
alice_status_check = Message(
    id = "unique-id-293e9a922efffxxx",
    type = "https://didcomm.org/messagepickup/3.0/status-request",
    body = {}
)

In [20]:
alice_status_check_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_status_check,
    to = mediator_did,
    frm = alice_did_for_mediator,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp3 = requests.post(mediator_endpoint, headers=headers, data = alice_status_check_packed_msg.packed_msg)

In [21]:
mediator_unpack_status = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp3.json()
)
print(mediator_unpack_status.message.type)
print("Messages in Mediator queue: " + str(mediator_unpack_status.message.body["message_count"]))

https://didcomm.org/messagepickup/3.0/status
Messages in Mediator queue: 0


### If there are messages, Alice can pickup from the mediator (delivery request)

In [22]:
alice_delivery_request = Message(
    id = "unique-id-293e9a922efffxxxff",
    type = "https://didcomm.org/messagepickup/3.0/delivery-request",
    body = {"limit": 1}
)

In [23]:
alice_delivery_request_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_delivery_request,
    to = mediator_did,
    frm = alice_did_for_mediator,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp4 = requests.post(mediator_endpoint, headers=headers, data = alice_delivery_request_packed_msg.packed_msg)

In [24]:
mediator_delivery = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp4.json()
)
print(mediator_delivery.message.type)
print(mediator_delivery.message.body)

https://didcomm.org/messagepickup/3.0/status
{'recipient_key': None, 'message_count': 0, 'live_delivery': False}


In [25]:
bob_packed_msg = mediator_delivery.message.attachments[0].data.json
msg_id = mediator_delivery.message.attachments[0].id

TypeError: 'NoneType' object is not subscriptable

In [ ]:
print(bob_packed_msg)

In [ ]:
bob_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= bob_packed_msg
)

In [ ]:
print("Message ID:", msg_id)
print(bob_msg.message.body["content"])

### Alice acknowledge so the mediator can delete the message

In [ ]:
alice_ack = Message(
    id = "unique-id-293e9a922efffxxxffsss",
    type = "https://didcomm.org/messagepickup/3.0/messages-received",
    body = {"message_id_list": [msg_id]}
)

In [ ]:
alice_ack_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_ack,
    to = mediator_did,
    frm = alice_did_for_mediator,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp5 = requests.post(mediator_endpoint, headers=headers, data = alice_ack_packed_msg.packed_msg)

In [ ]:
mediator_ack_status = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp5.json()
)
print(mediator_ack_status.message.type)
print(mediator_ack_status.message.body)